In [1]:
#Bibliotecas necesarias :

#!pip install pyspark kafka-python pandas matplotlib

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, window, desc, lag, avg, stddev, min, max, sum, approx_count_distinct
from pyspark.sql.window import Window
from kafka import KafkaProducer
from kafka import KafkaConsumer

import os
import csv
import random
import datetime
import json
import time

In [33]:

# Configura tu Kafka
KAFKA_TOPIC = 'logs'
KAFKA_BROKER = 'localhost:9093'  # Cambia esto si usas Docker o red externa

# Inicializar el productor
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Tipos de eventos
event_types = [
    ('INFO', 0.6),
    ('WARNING', 0.15),
    ('ERROR', 0.1),
    ('DEBUG', 0.1),
    ('LOGIN_FAILURE', 0.05)
]

users = ['admin', 'user1', 'user2', 'guest', 'root', 'test']
ips = [
    '192.168.1.10', '192.168.1.15', '10.0.0.1', '172.16.0.2',
    '203.0.113.45', '8.8.8.8', '185.199.108.153'
]

def choose_event_type():
    return random.choices([e[0] for e in event_types], weights=[e[1] for e in event_types])[0]

def generate_log():
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    event_type = choose_event_type()
    user = random.choice(users)
    ip = random.choice(ips)

    if event_type == 'LOGIN_FAILURE':
        message = f"Intento de acceso fallido para usuario {user} desde {ip}"
    elif event_type == 'ERROR':
        message = f"Error critico en el sistema detectado por el usuario {user}"
    elif event_type == 'WARNING':
        message = f"Posible anomalia detectada desde la IP {ip}"
    elif event_type == 'DEBUG':
        message = f"Modo depuración activo por usuario {user}"
    else:
        message = f"Evento normal registrado para usuario {user} desde {ip}"

    return {
        'timestamp': timestamp,
        'event_type': event_type,
        'user': user,
        'ip': ip,
        'message': message
    }

# Bucle infinito para enviar logs
try:
    print(f"Enviando logs a Kafka topic '{KAFKA_TOPIC}'...")
    while True:
        log = generate_log()
        producer.send(KAFKA_TOPIC, value=log)
        #print("Enviado:", log)
        time.sleep(1)  # Esperar 1 segundo entre logs (ajustable)
except KeyboardInterrupt:
    print("Interrumpido por el usuario")
finally:
    producer.close()


Enviando logs a Kafka topic 'logs'...
Interrumpido por el usuario
